In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import pandas as pd

# Read the CSV file
df = pd.read_csv('/New_Australian_EV_Charging_Stations.csv')


In [6]:
import folium

# Create a base map
base_map = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=10)

# Add markers for EV charging stations
for idx, row in df.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], popup=row['Location Name']).add_to(base_map)

# Show the map
base_map



## Google MAPs API to get traffic data

In [3]:
import googlemaps
from datetime import datetime

# Initialize the Google Maps API client
gmaps = googlemaps.Client(key='FILL IN YOUR OWN API')

# Define your coordinates
latitude = -35.72671957
longitude = 145.6593541


now = datetime.now()


traffic_model = "optimistic"  

# Use the `directions` method to get driving directions between points
directions_result = gmaps.directions(f"{latitude},{longitude}",
                                     "28 William St, Cobram VIC 3644",
                                     mode="driving",
                                     departure_time=now,
                                     traffic_model=traffic_model)

# Extract and print the estimated duration in traffic
try:
    print("Estimated time in traffic: ", directions_result[0]['legs'][0]['duration_in_traffic']['text'])
except KeyError:
    print("Could not get the estimated time in traffic.")


Estimated time in traffic:  22 mins


## Geocoding (Address to Coordinates)

In [4]:
import requests
import json

def geocode(address, api_key):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}"
    response = requests.get(url)
    data = response.json()
    if data['status'] == 'OK':
        latitude = data['results'][0]['geometry']['location']['lat']
        longitude = data['results'][0]['geometry']['location']['lng']
        return latitude, longitude
    else:
        return None, None

api_key = "FILL IN YOUR OWN API"
address = "Weddell Road, North Geelong, Vic, 3215"

latitude, longitude = geocode(address, api_key)
print(f"Latitude: {latitude}, Longitude: {longitude}")


Latitude: -38.126969, Longitude: 144.3461291


## Reverse Geocoding (Coordinates to Address)

In [5]:
def reverse_geocode(latitude, longitude, api_key):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={latitude},{longitude}&key={api_key}"
    response = requests.get(url)
    data = response.json()
    if data['status'] == 'OK':
        address = data['results'][0]['formatted_address']
        return address
    else:
        return None


api_key = "FILL IN YOUR OWN API"
latitude = -38.129308
longitude = 144.345207

address = reverse_geocode(latitude, longitude, api_key)
print(f"Address: {address}")


Address: 2 The Fairway, North Geelong VIC 3215, Australia


In [411]:
import requests
import json

def search_places_nearby(latitude, longitude, keyword, api_key):
    # Define the endpoint URL
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={latitude},{longitude}&radius=10&keyword={keyword}&key={api_key}"
    
    # Make the GET request
    response = requests.get(url)
    
    # Convert the response to a dictionary
    data = response.json()
    
    # Check if the request was successful
    if data['status'] == 'OK':
        # Loop through the results and print the place names
        for place in data['results']:
            print(f"Place name: {place['name']}")
            print(f"Place address: {place.get('vicinity', 'N/A')}")
            print("-----")
    else:
        print(f"Error fetching data: {data['status']}")

api_key = "FILL IN YOUR OWN API"

# Example usage
search_places_nearby(-38.338316,144.743447, "coffee", api_key)


Place name: Rusty's Cafe Bar and Grill
Place address: 3293 Point Nepean Rd, Sorrento
-----
Place name: The Vanilla Slice Cafe Sorrento
Place address: 23 Ocean Beach Rd, Sorrento
-----
Place name: Stringers Sorrento
Place address: 2-8 Ocean Beach Rd, Sorrento
-----
